1. K-Means
2. Fuzzy C-means or EM
4. DBScan
5. Spectral

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/Shareddrives/CSCI\ 5523\ -\ Data\ Mining\ Final\ Project
!ls

# import datasets
df_labels=pd.read_csv('midterm-2018.tsv', sep='\t')
df_labels=df_labels.rename(columns={'2521260264':'user_id'})

df_users=pd.read_json('midterm-2018_processed_user_objects.json')

# Merge our user objects and labels.
df=pd.merge(df_users, df_labels, on=['user_id'])

# Look at the shape and information
print(df.shape)
print(df.info())

# check bot vs human counts
print(df['bot'].value_counts())

# create unbalanced train and test dataframes
train_u, test_u = train_test_split(df, test_size=0.2)

# equal datatypes check
if all(train_u.dtypes == test_u.dtypes):
  print("pass: data types are all same between train and test")
else:
  print('\nFAIL: DATA TYPES ARE NOT EQUAL BETWEEN TRAIN AND TEST, SOMETHING IS WRONG\n')


# create balanced train and test datframes
bots=df[df['bot']=='bot']
humans=df[df['bot']=='human']
bots_sample=bots.sample(8092)
bal_df=bots_sample.append(humans)

train_b, test_b = train_test_split(bal_df, test_size=0.2)

# equal datatypes check
if all(train_b.dtypes == test_b.dtypes):
  print("pass: data types are all same between train and test")
else:
  print('\nFAIL: DATA TYPES ARE NOT EQUAL BETWEEN TRAIN AND TEST, SOMETHING IS WRONG\n')



dataframes = [train_u, test_u, train_b, test_b]

# clean dataframes
cols_to_drop = ['probe_timestamp', 'user_id', 'screen_name', 'name', 'description', 'user_created_at', 'url', 'lang', 'protected', 'tid']

for dataframe in dataframes:
  dataframe.drop(labels=cols_to_drop, axis=1, inplace=True)                 # drop unused columns
  dataframe['bot'] = [1 if val=='bot' else 0 for val in dataframe['bot']]   # convert bot column from string to int (1 for bot, 0 for human)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/Shareddrives/CSCI 5523 - Data Mining Final Project'
/content/gdrive/Shareddrives/CSCI 5523 - Data Mining Final Project
 Balance_deprecated.ipynb   classification.ipynb   data
'Bot Detection.gslides'     cluster_2.ipynb	   data.ipynb
 botDetection.mp4	    cluster.ipynb	   Proposal.gdoc


FileNotFoundError: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train_u = pd.read_json('data/more_human_train.json')

In [ ]:
test_u = pd.read_json('data/more_human_test.json')

In [ ]:
train_b = pd.read_json('data/bal_train.json')

In [ ]:
test_b = pd.read_json('data/bal_test.json')

In [ ]:
dataframes = [train_u, test_u, train_b, test_b]

for dataframe in dataframes:
  dataframe['bot'] = [1 if val=='bot' else 0 for val in dataframe['bot']]   # convert bot column from string to int (1 for bot, 0 for human)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def print_analysis(test, predictions, label):
  print("-----------------------------------")
  print('{} Accuracy score: {}'.format(label, accuracy_score(test, predictions)))
  print('{} Precision score: {}'.format(label, precision_score(test, predictions)))
  print('{} Recall score: {}'.format(label, recall_score(test, predictions)))
  print('{} F1 score: {}'.format(label, f1_score(test, predictions)))
  print("-----------------------------------\n")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# UNBALANCED
x_train_u = train_u.to_numpy()[:,:-1].astype(int)   # all cols of train_u except for class label
x_train_u_scaled = scaler.fit_transform(x_train_u)

y_train_u = train_u.to_numpy()[:,-1].astype(int)    # only class label (last) column of train_u

x_test_u = test_u.to_numpy()[:,:-1].astype(int)   # all cols of test_u except for class label
x_test_u_scaled = scaler.transform(x_test_u)

y_test_u = test_u.to_numpy()[:,-1].astype(int)    # only class label (last) column of test_u

# BALANCED
x_train_b = train_b.to_numpy()[:,:-1].astype(int)   # all cols of train_b except for class label
x_train_b_scaled = scaler.fit_transform(x_train_b)

y_train_b = train_b.to_numpy()[:,-1].astype(int)    # only class label (last) column of train_b

x_test_b = test_b.to_numpy()[:,:-1].astype(int)   # all cols of test_b except for class label
x_test_b_scaled = scaler.transform(x_test_b)

y_test_b = test_b.to_numpy()[:,-1].astype(int)    # only class label (last) column of test_b

##Density Based Clustering: DBScan

Unlike K-Means, DBScan does not have a predict function as it is not intended for classified learning (no centroids).  We can still learn things from its cluster sizes and entropy (homogeneity) however.

###DBScan on unbalanced data

In [ ]:
from sklearn import cluster
from sklearn.cluster import DBSCAN

data = pd.DataFrame(x_train_u_scaled, columns=train_u.columns[:-1])
db = DBSCAN(eps=0.005, min_samples=50).fit(data)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = pd.DataFrame(db.labels_,columns=['Cluster ID'])
pd.DataFrame(labels, columns=['Cluster ID'])


,Cluster ID
0,-1
1,-1
2,-1
3,-1
4,-1
...,...
7794,-1
7795,-1
7796,-1
7797,-1


###Analysis

Compare cluster sizes with class sizes

In [ ]:
np.unique(pd.DataFrame(labels, columns=['Cluster ID']), return_counts=True)

(array([-1,  0]), array([6499, 1300]))

In [ ]:
np.unique(y_train_u, return_counts=True)

(array([0, 1]), array([6469, 1330]))

Find homogeneity completeness and v-measure scores (measurements of entropy). 1 is perfect clustering of labels, 0 is worst case.

In [ ]:
from sklearn import metrics
# Evaluate on training data
np.unique(y_train_u, return_counts=True)
#np.shape(y_train_u)
#np.shape(labels)
metrics.homogeneity_completeness_v_measure(y_train_u, np.ndarray.flatten((labels.to_numpy())))

(0.5755017987637634, 0.5833405123464755, 0.579394643996971)

###DBScan on balanced data

In [ ]:
from sklearn.cluster import DBSCAN

data = pd.DataFrame(x_train_b_scaled, columns=train_b.columns[:-1])
db = DBSCAN(eps=0.05, min_samples=50).fit(data)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = pd.DataFrame(db.labels_,columns=['Cluster ID'])
pd.DataFrame(labels, columns=['Cluster ID'])




,Cluster ID
0,-1
1,0
2,0
3,0
4,-1
...,...
12942,2
12943,0
12944,0
12945,0


###Analysis

Compare cluster sizes with class sizes

In [ ]:
np.unique(pd.DataFrame(labels, columns=['Cluster ID']), return_counts=True)

(array([-1,  0,  1,  2,  3,  4,  5]),
 array([4991, 6824,  407,  275,  179,  135,  136]))

In [ ]:
np.unique(y_train_b, return_counts=True)

(array([0, 1]), array([6509, 6438]))

Find homogeneity completeness and v-measure scores (measurements of entropy). 1 is perfect clustering of labels, 0 is worst case.

In [ ]:
from sklearn import metrics
# Evaluate on training data
np.unique(y_train_b, return_counts=True)
#np.shape(y_train_u)
#np.shape(labels)
metrics.homogeneity_completeness_v_measure(y_train_b, np.ndarray.flatten((labels.to_numpy())))


(0.6661943006476313, 0.43968063614706737, 0.5297393478421498)

##Spectral Clustering

###Spectral Clustering on unbalanced data
####Takes a very long time

Remove outlier

In [ ]:
# train_u_new = train_u.drop(train_u.index[17007])
# x_train_u_new = train_u_new.to_numpy()[:,:-1].astype(int)
# x_train_u_new_scaled = scaler.fit_transform(x_train_u_new)
# y_train_u_new = train_u_new.to_numpy()[:,-1].astype(int)

# x_test_u_new_scaled = scaler.transform(x_test_u)

In [ ]:
data = pd.DataFrame(x_train_u_scaled, columns=train_u.columns[:-1])

spectral = cluster.SpectralClustering(n_clusters=8,random_state=1,affinity='nearest_neighbors',gamma=100)
spectral.fit(data)
labels = pd.DataFrame(spectral.labels_,columns=['Cluster ID'])
#result = pd.concat((data1,labels1), axis=1)


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


###Analysis

Compare cluster sizes with class sizes

In [ ]:
np.unique(pd.DataFrame(labels, columns=['Cluster ID']), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32),
 array([2310, 3882,   11, 1480,   27,   20,   22,   47]))

In [ ]:
np.unique(y_train_u, return_counts=True)

(array([0, 1]), array([6469, 1330]))

Find homogeneity completeness and v-measure scores (measurements of entropy). 1 is perfect clustering of labels, 0 is worst case.

In [ ]:
from sklearn import metrics
# Evaluate on training data
np.unique(y_train_u, return_counts=True)
#np.shape(y_train_u)
#np.shape(labels)
metrics.homogeneity_completeness_v_measure(y_train_u, np.ndarray.flatten((labels.to_numpy())))

(0.4439873823588762, 0.181938761215739, 0.25810877264359916)

###Spectral Clustering on balanced data
####Takes a very long time

Remove outlier

In [ ]:
# train_b_new = train_b.drop(train_b.index[3798])
# x_train_b_new = train_b_new.to_numpy()[:,:-1].astype(int)
# x_train_b_new_scaled = scaler.fit_transform(x_train_b_new)
# y_train_b_new = train_b_new.to_numpy()[:,-1].astype(int)

# x_test_b_new_scaled = scaler.transform(x_test_b)

In [ ]:
data = pd.DataFrame(x_train_b_scaled, columns=train_b.columns[:-1])

spectral = cluster.SpectralClustering(n_clusters=8,random_state=1,affinity='nearest_neighbors',gamma=100)
spectral.fit(data)
labels = pd.DataFrame(spectral.labels_,columns=['Cluster ID'])
#result = pd.concat((data1,labels1), axis=1)


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


###Analysis

Compare cluster sizes with class sizes

In [ ]:
np.unique(pd.DataFrame(labels, columns=['Cluster ID']), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32),
 array([   47, 12282,    16,    43,   185,   104,    76,   194]))

In [ ]:
np.unique(y_train_b, return_counts=True)

(array([0, 1]), array([6509, 6438]))

Find homogeneity completeness and v-measure scores (measurements of entropy). 1 is perfect clustering of labels, 0 is worst case.

In [ ]:
from sklearn import metrics
# Evaluate on training data
np.unique(y_train_b, return_counts=True)
#np.shape(y_train_u)
#np.shape(labels)
metrics.homogeneity_completeness_v_measure(y_train_b, np.ndarray.flatten((labels.to_numpy())))

(0.053801518079819755, 0.12849950095181228, 0.07584673152602785)